<a href="https://colab.research.google.com/github/achmadindra/Tugas_Kelompok_Pengenalan-Wajah-dengan-Deepface/blob/main/main_program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tugas Pengenalan Wajah dengan Deepface**

**Proses menambahkan drive dimana file dataset disimpan.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


**Proses menginstall library Deepface.**

In [ ]:
pip install deepface

     |████████████████████████████████| 65 kB 2.3 MB/s 
     |████████████████████████████████| 87 kB 4.4 MB/s 
     |████████████████████████████████| 60.5 MB 1.2 MB/s 
     |████████████████████████████████| 2.3 MB 33.4 MB/s 
     |████████████████████████████████| 462 kB 46.7 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=2e482519ee5b4ccbe847d8b4210bc9f9a77d7f01d11c75428999f4a29641b8e2
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have im

**Proses penambahan library lain yang dibutuhkan program.**

In [ ]:
#Library Deepface
from deepface import DeepFace
#Library untuk mengolah data
import pandas as pd
from IPython.core.display import display,HTML
import os
import pathlib

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


**Proses utama pengenalan wajah dengan library Deepface.**

Di bagian ini, dibuat fungsi yang bertugas untuk mengenali wajah menggunakan fungsi `find` pada library Deepface. `find` dapat membandingkan suatu gambar (foto) masukan dengan kumpulan foto pada dataset. Pembandingan dilakukan dengan menghitung 'jarak' perbedaan antara 2 foto yang dibandingkan. Semakin kecil nilai dari 'jarak' ini, wajah dalam foto akan semakin mirip.

Di fungsi `cekWajah`, diberikan *threshold* dari nilai 'jarak' ini. Apabila semua 'jarak' dari wajah yang ada di dataset dengan wajah masukan, dianggap wajah masukan tersebut 'unknown'.

Selain itu, di fungsi ini juga dilakukan pengolahan string nama dari file wajah di dataset yang paling mirip dengan wajah masukan.

In [ ]:
def cekWajah(image,model,threshold):
  #Proses membandingkan wajah (menghitung 'jarak') masukan dengan wajah-wajah di database
  df=DeepFace.find(img_path="/content/drive/MyDrive/tugasFR/tes/"+image, db_path="/content/drive/MyDrive/tugasFR/Kelas", model_name = model, distance_metric = 'cosine', model = None, enforce_detection = False, detector_backend = 'dlib', align = True, prog_bar = True, normalization = 'base')
  
  #Proses mengolah data hasil pembandingan
  for i in range (df.shape[0]):

    if (model=='VGG-Face'):
      distance = 'VGG-Face_cosine'
    elif (model == 'Facenet'):
      distance = 'Facenet_cosine'
    elif (model=='Dlib'):
      distance = 'Dlib_cosine'
    elif (model=='ArcFace'):
      distance = 'ArcFace_cosine'
    #Mengolah string nama file dari wajah di dataset yang memiliki 'jarak' kurang dari threshold yang ditentukan
    if(df[distance][i])<threshold:
      kalimat=df['identity'][i]
      panjang=len(kalimat)
      posSlash=kalimat.rfind('/', 0, panjang)
      lensubstring=panjang-posSlash
      nmFile=kalimat[posSlash+1:panjang]
      posTitik=nmFile.rfind('.', 0, lensubstring)
      lenFile=len(nmFile)
      posTipe=lenFile-(posTitik)
      found=nmFile[0:lenFile-posTipe]
      return kalimat, found
      break
    else:
      break


**Proses pengujian algoritma pengenalan wajah di library Deepface.**

Di sini, digunakan fungsi `cekWajah` yang telah dibuat terhadap test set yang sebelumnya telah di-list.

# Percobaan dengan Model VGG-Face

In [ ]:
dataresult=[]
#Variabel penyimpan path untuk penampilan gambar
visual=[]
#Variabel untuk menyimpan banyak wajah yang dideteksi dengan benar
counter=0

#File untuk menyimpan hasil deteksi wajah
f = open("/content/drive/MyDrive/tugasFR/hasil.txt", "w")

#Menentukan label dari tiap wajah di test set
for filename in os.listdir('/content/drive/MyDrive/tugasFR/tes'):
  ext = [".jpg", ".jpeg", ".png", ".jfif"]
  if filename.endswith(tuple(ext)):
    #Menggunakan fungsi cekWajah untuk menentukan wajah di dataset yang paling mirip dengan wajah masukan
    ketemu=cekWajah(image=filename,model='VGG-Face',threshold=0.27)
    sumber=filename
    substrsumber=sumber[0:7]

    #Menentukan apakah hasil pelabelan wajah masukan sudah sama dengan label wajah di dataset
    if ketemu:
      target=ketemu[1]
      substrtarget=target[0:7]
      if substrsumber==substrtarget:
        sama="Benar"
        counter=counter+1
      else:
        sama="Salah"
      #Memasukkan hasil deteksi ke hasil-hasil sebelumnya
      dataresult.append([pathlib.Path(sumber).stem,target,sama])
      visual.append([sumber,ketemu[0],sama])
    else:
      target='unknown'
      print(substrsumber)
      if substrsumber==target:
        sama="Benar"
        counter=counter+1
      else:
        sama="Salah"
      #Memasukkan hasil deteksi ke hasil-hasil sebelumnya
      dataresult.append([pathlib.Path(sumber),target,sama])
      visual.append([sumber,'/content/drive/MyDrive/tugasFR/unknown.png',sama])

    #File penyimpan untuk troubleshoot hasil
    f.write('sumber = '+sumber+' Target = '+target+' '+sama+'\n')
    print('sumber = '+sumber+' Target = '+target+' '+sama)

f.close()  

There are  42  representations found in  representations_vgg_face.pkl
find function lasts  0.46552371978759766  seconds
sumber = hartanto tantriawan(1).jfif Target = hartanto tantriawan 1  Benar
There are  42  representations found in  representations_vgg_face.pkl
find function lasts  1.9033334255218506  seconds
sumber = meredita susanty(1).jpg Target = meredita susanty Benar
There are  42  representations found in  representations_vgg_face.pkl
find function lasts  6.567559242248535  seconds
sumber = arief sartono 15.jpg Target = arief sartono GB Benar
There are  42  representations found in  representations_vgg_face.pkl
find function lasts  0.3464639186859131  seconds
sumber = hartanto tantriawan.jfif Target = hartanto tantriawan 1  Benar
There are  42  representations found in  representations_vgg_face.pkl
find function lasts  6.308821201324463  seconds
sumber = arief sartono 10.jpg Target = arief sartono Benar
There are  42  representations found in  representations_vgg_face.pkl
fin

**Proses Penampilan dan Penyimpanan Data**

In [ ]:
hasil=pd.DataFrame(dataresult)
hasil.columns = ["Label Test Data", "Label Database", "Hasil Deteksi"]
visual_path=pd.DataFrame(visual)

In [ ]:
print(hasil)
print(visual_path)
hasil.to_csv('/content/drive/MyDrive/tugasFR/Hasil CSV/VGG_Face.csv')

             Label Test Data          Label Database Hasil Deteksi
0     hartanto tantriawan(1)  hartanto tantriawan 1          Benar
1        meredita susanty(1)        meredita susanty         Benar
2           arief sartono 15        arief sartono GB         Benar
3        hartanto tantriawan  hartanto tantriawan 1          Benar
4           arief sartono 10           arief sartono         Benar
..                       ...                     ...           ...
101       baud prananto6.jpg                 unknown         Salah
102  hartanto tantriawan (1)  hartanto tantriawan 1          Benar
103      dewi tresnawati (1)         dewi tresnawati         Benar
104        imam ekowicaksono          handoko supeno         Salah
105             meza silvana         meza silvana GB         Benar

[106 rows x 3 columns]
                               0  \
0    hartanto tantriawan(1).jfif   
1        meredita susanty(1).jpg   
2           arief sartono 15.jpg   
3       hartanto tantriawan.

In [ ]:
#Perhitungan akurasi
akurasi=counter/len(hasil)*100
print(akurasi)

61.32075471698113


**Proses Menampilkan Hasil dengan Gambar**

In [ ]:
#Library gambar
from IPython.display import Image, display

#Proses menampilkan gambar untuk tiap gambar yang dideteksi
for i in range(len(visual_path)):
  if((pathlib.Path(visual_path[0][i]).suffix != '.jfif') and (pathlib.Path(visual_path[1][i]).suffix != '.jfif')):
    print(i)
    pathsumber='/content/drive/MyDrive/tugasFR/tes/'+visual_path[0][i]
    labelsumber=visual_path[0][i]
    display(Image(filename=pathsumber,width = 100, height = 100))
    print(pathlib.Path(labelsumber).stem)
    pathtarget=visual_path[1][i]
    labeltarget=visual_path[1][i]
    display(Image(filename=pathtarget,width = 100, height = 100))
    print(pathlib.Path(labeltarget).stem)
    print(hasil["Hasil Deteksi"][i])
    print('-------------------------------------------------------')

# Percobaan dengan Model FaceNet

In [ ]:
dataresult=[]
#Variabel penyimpan path untuk penampilan gambar
visual=[]
#Variabel untuk menyimpan banyak wajah yang dideteksi dengan benar
counter=0

#File untuk menyimpan hasil deteksi wajah
f = open("/content/drive/MyDrive/tugasFR/hasilFaceNet.txt", "w")

#Menentukan label dari tiap wajah di test set
for filename in os.listdir('/content/drive/MyDrive/tugasFR/tes'):
  ext = [".jpg", ".jpeg", ".png", ".jfif"]
  if filename.endswith(tuple(ext)):
    #Menggunakan fungsi cekWajah untuk menentukan wajah di dataset yang paling mirip dengan wajah masukan
    ketemu=cekWajah(image=filename,model='Facenet',threshold=0.5)
    sumber=filename
    substrsumber=sumber[0:7]

    #Menentukan apakah hasil pelabelan wajah masukan sudah sama dengan label wajah di dataset
    if ketemu:
      target=ketemu[1]
      substrtarget=target[0:7]
      if substrsumber==substrtarget:
        sama="Benar"
        counter=counter+1
      else:
        sama="Salah"
      #Memasukkan hasil deteksi ke hasil-hasil sebelumnya
      dataresult.append([pathlib.Path(sumber).stem,target,sama])
      visual.append([sumber,ketemu[0],sama])
    else:
      target='unknown'
      print(substrsumber)
      if substrsumber==target:
        sama="Benar"
        counter=counter+1
      else:
        sama="Salah"
      #Memasukkan hasil deteksi ke hasil-hasil sebelumnya
      dataresult.append([pathlib.Path(sumber).stem,target,sama])
      visual.append([sumber,'/content/drive/MyDrive/tugasFR/unknown.png',sama])

    #File penyimpan untuk troubleshoot hasil
    f.write('sumber = '+sumber+' Target = '+target+' '+sama+'\n')
    print('sumber = '+sumber+' Target = '+target+' '+sama)

f.close()  

There are  42  representations found in  representations_facenet.pkl
find function lasts  0.24889898300170898  seconds
sumber = hartanto tantriawan(1).jfif Target = hartanto tantriawan 1  Benar
There are  42  representations found in  representations_facenet.pkl
find function lasts  0.6155638694763184  seconds
sumber = meredita susanty(1).jpg Target = meredita susanty Benar
There are  42  representations found in  representations_facenet.pkl
find function lasts  6.098846673965454  seconds
sumber = arief sartono 15.jpg Target = arief sartono Benar
There are  42  representations found in  representations_facenet.pkl
find function lasts  0.17720913887023926  seconds
sumber = hartanto tantriawan.jfif Target = hartanto tantriawan 1  Benar
There are  42  representations found in  representations_facenet.pkl
find function lasts  6.100850820541382  seconds
sumber = arief sartono 10.jpg Target = arief sartono Benar
There are  42  representations found in  representations_facenet.pkl
find functi

**Proses Penampilan dan Penyimpanan Data**

In [ ]:
hasil=pd.DataFrame(dataresult)
print(hasil)
hasil.columns = ["Label Test Data", "Label Database", "Hasil Deteksi"]
hasil.to_csv('/content/drive/MyDrive/tugasFR/Hasil CSV/FaceNet.csv')
#Perhitungan akurasi
akurasi=counter/len(hasil)*100
print("akurasi: "+ str(akurasi))


                           0                       1      2
0     hartanto tantriawan(1)  hartanto tantriawan 1   Benar
1        meredita susanty(1)        meredita susanty  Benar
2           arief sartono 15           arief sartono  Benar
3        hartanto tantriawan  hartanto tantriawan 1   Benar
4           arief sartono 10           arief sartono  Benar
..                       ...                     ...    ...
101           baud prananto6                 unknown  Salah
102  hartanto tantriawan (1)  hartanto tantriawan 1   Benar
103      dewi tresnawati (1)         dewi tresnawati  Benar
104        imam ekowicaksono    imam ekowicaksono GB  Benar
105             meza silvana         meza silvana GB  Benar

[106 rows x 3 columns]
akurasi: 81.13207547169812


**Proses Menampilkan Hasil dengan Gambar**

In [ ]:
visual_path=pd.DataFrame(visual)
#Proses menampilkan gambar untuk tiap gambar yang dideteksi
for i in range(len(visual_path)):
  if((pathlib.Path(visual_path[0][i]).suffix != '.jfif') and (pathlib.Path(visual_path[1][i]).suffix != '.jfif')):
    print(i)
    pathsumber='/content/drive/MyDrive/tugasFR/tes/'+visual_path[0][i]
    labelsumber=visual_path[0][i]
    display(Image(filename=pathsumber,width = 100, height = 100))
    print(pathlib.Path(labelsumber).stem)
    pathtarget=visual_path[1][i]
    labeltarget=visual_path[1][i]
    display(Image(filename=pathtarget,width = 100, height = 100))
    print(pathlib.Path(labeltarget).stem)
    print(hasil["Hasil Deteksi"][i])
    print('-------------------------------------------------------')

# Percobaan dengan Model Dlib

In [ ]:
dataresult=[]
#Variabel penyimpan path untuk penampilan gambar
visual=[]
#Variabel untuk menyimpan banyak wajah yang dideteksi dengan benar
counter=0

#File untuk menyimpan hasil deteksi wajah
f = open("/content/drive/MyDrive/tugasFR/hasilDlib.txt", "w")

#Menentukan label dari tiap wajah di test set
for filename in os.listdir('/content/drive/MyDrive/tugasFR/tes'):
  ext = [".jpg", ".jpeg", ".png", ".jfif"]
  if filename.endswith(tuple(ext)):
    #Menggunakan fungsi cekWajah untuk menentukan wajah di dataset yang paling mirip dengan wajah masukan
    ketemu=cekWajah(image=filename,model='Dlib',threshold=0.08)
    sumber=filename
    substrsumber=sumber[0:7]

    #Menentukan apakah hasil pelabelan wajah masukan sudah sama dengan label wajah di dataset
    if ketemu:
      target=ketemu[1]
      substrtarget=target[0:7]
      if substrsumber==substrtarget:
        sama="Benar"
        counter=counter+1
      else:
        sama="Salah"
      #Memasukkan hasil deteksi ke hasil-hasil sebelumnya
      dataresult.append([pathlib.Path(sumber).stem,target,sama])
      visual.append([sumber,ketemu[0],sama])
    else:
      target='unknown'
      print(substrsumber)
      if substrsumber==target:
        sama="Benar"
        counter=counter+1
      else:
        sama="Salah"
      #Memasukkan hasil deteksi ke hasil-hasil sebelumnya
      dataresult.append([pathlib.Path(sumber).stem,target,sama])
      visual.append([sumber,'/content/drive/MyDrive/tugasFR/unknown.png',sama])

    #File penyimpan untuk troubleshoot hasil
    f.write('sumber = '+sumber+' Target = '+target+' '+sama+'\n')
    print('sumber = '+sumber+' Target = '+target+' '+sama)

f.close()  

There are  42  representations found in  representations_dlib.pkl
find function lasts  0.1729869842529297  seconds
sumber = hartanto tantriawan(1).jfif Target = hartanto tantriawan 1  Benar
There are  42  representations found in  representations_dlib.pkl
find function lasts  0.5052742958068848  seconds
sumber = meredita susanty(1).jpg Target = meredita susanty Benar
There are  42  representations found in  representations_dlib.pkl
find function lasts  6.062509536743164  seconds
sumber = arief sartono 15.jpg Target = arief sartono Benar
There are  42  representations found in  representations_dlib.pkl
find function lasts  0.09698009490966797  seconds
sumber = hartanto tantriawan.jfif Target = hartanto tantriawan 1  Benar
There are  42  representations found in  representations_dlib.pkl
find function lasts  6.005103349685669  seconds
sumber = arief sartono 10.jpg Target = arief sartono Benar
There are  42  representations found in  representations_dlib.pkl
find function lasts  6.0846850

**Proses Penampilan dan Penyimpanan Data**

In [ ]:
hasil=pd.DataFrame(dataresult)
print(hasil)
hasil.columns = ["Label Test Data", "Label Database", "Hasil Deteksi"]
hasil.to_csv('/content/drive/MyDrive/tugasFR/Hasil CSV/Dlib.csv')
#Perhitungan akurasi
akurasi=counter/len(hasil)*100
print("akurasi: "+ str(akurasi))

                           0                                 1      2
0     hartanto tantriawan(1)            hartanto tantriawan 1   Benar
1        meredita susanty(1)                  meredita susanty  Benar
2           arief sartono 15                     arief sartono  Benar
3        hartanto tantriawan            hartanto tantriawan 1   Benar
4           arief sartono 10                     arief sartono  Benar
..                       ...                               ...    ...
101           baud prananto6  kemas muhammad irsan riza (1) GB  Salah
102  hartanto tantriawan (1)            hartanto tantriawan 1   Benar
103      dewi tresnawati (1)                   dewi tresnawati  Benar
104        imam ekowicaksono              imam ekowicaksono GB  Benar
105             meza silvana                   meza silvana GB  Benar

[106 rows x 3 columns]
akurasi: 75.47169811320755


**Proses Menampilkan Hasil dengan Gambar**

In [ ]:
visual_path=pd.DataFrame(visual)
#Proses menampilkan gambar untuk tiap gambar yang dideteksi
for i in range(len(visual_path)):
  if((pathlib.Path(visual_path[0][i]).suffix != '.jfif') and (pathlib.Path(visual_path[1][i]).suffix != '.jfif')):
    print(i)
    pathsumber='/content/drive/MyDrive/tugasFR/tes/'+visual_path[0][i]
    labelsumber=visual_path[0][i]
    display(Image(filename=pathsumber,width = 100, height = 100))
    print(pathlib.Path(labelsumber).stem)
    pathtarget=visual_path[1][i]
    labeltarget=visual_path[1][i]
    display(Image(filename=pathtarget,width = 100, height = 100))
    print(pathlib.Path(labeltarget).stem)
    print(hasil["Hasil Deteksi"][i])
    print('-------------------------------------------------------')

# Percobaan dengan Model ArcFace

In [ ]:
dataresult=[]
#Variabel penyimpan path untuk penampilan gambar
visual=[]
#Variabel untuk menyimpan banyak wajah yang dideteksi dengan benar
counter=0

#File untuk menyimpan hasil deteksi wajah
f = open("/content/drive/MyDrive/tugasFR/hasilArcFace.txt", "w")

#Menentukan label dari tiap wajah di test set
for filename in os.listdir('/content/drive/MyDrive/tugasFR/tes'):
  ext = [".jpg", ".jpeg", ".png", ".jfif"]
  if filename.endswith(tuple(ext)):
    #Menggunakan fungsi cekWajah untuk menentukan wajah di dataset yang paling mirip dengan wajah masukan
    ketemu=cekWajah(image=filename,model='ArcFace',threshold=0.55)
    sumber=filename
    substrsumber=sumber[0:7]

    #Menentukan apakah hasil pelabelan wajah masukan sudah sama dengan label wajah di dataset
    if ketemu:
      target=ketemu[1]
      substrtarget=target[0:7]
      if substrsumber==substrtarget:
        sama="Benar"
        counter=counter+1
      else:
        sama="Salah"
        #Memasukkan hasil deteksi ke hasil-hasil sebelumnya
      dataresult.append([pathlib.Path(sumber).stem,target,sama])
      visual.append([sumber,ketemu[0],sama])
    else:
      target='unknown'
      print(substrsumber)
      if substrsumber==target:
        sama="Benar"
        counter=counter+1
      else:
        sama="Salah"
      #Memasukkan hasil deteksi ke hasil-hasil sebelumnya
      dataresult.append([pathlib.Path(sumber).stem,target,sama])
      visual.append([sumber,'/content/drive/MyDrive/tugasFR/unknown.png',sama])

    #File penyimpan untuk troubleshoot hasil
    f.write('sumber = '+sumber+' Target = '+target+' '+sama+'\n')
    print('sumber = '+sumber+' Target = '+target+' '+sama)

f.close()  

arcface_weights.h5  will be downloaded to  /root/.deepface/weights/arcface_weights.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/arcface_weights.h5
To: /root/.deepface/weights/arcface_weights.h5
100%|██████████| 137M/137M [00:01<00:00, 90.4MB/s]


There are  42  representations found in  representations_arcface.pkl
find function lasts  5.770447015762329  seconds
sumber = hartanto tantriawan(1).jfif Target = hartanto tantriawan 1  Benar
There are  42  representations found in  representations_arcface.pkl
find function lasts  0.5790491104125977  seconds
sumber = meredita susanty(1).jpg Target = meredita susanty Benar
There are  42  representations found in  representations_arcface.pkl
find function lasts  6.172733783721924  seconds
sumber = arief sartono 15.jpg Target = arief sartono Benar
There are  42  representations found in  representations_arcface.pkl
find function lasts  0.1878361701965332  seconds
sumber = hartanto tantriawan.jfif Target = hartanto tantriawan 1  Benar
There are  42  representations found in  representations_arcface.pkl
find function lasts  6.098533868789673  seconds
sumber = arief sartono 10.jpg Target = arief sartono Benar
There are  42  representations found in  representations_arcface.pkl
find function 

**Proses Penampilan dan Penyimpanan Data**

In [ ]:
hasil=pd.DataFrame(dataresult)
print(hasil)
hasil.columns = ["Label Test Data", "Label Database", "Hasil Deteksi"]
hasil.to_csv('/content/drive/MyDrive/tugasFR/Hasil CSV/ArcFace.csv')
#Perhitungan akurasi
akurasi=counter/len(hasil)*100
print("akurasi: "+ str(akurasi))

                           0                       1      2
0     hartanto tantriawan(1)  hartanto tantriawan 1   Benar
1        meredita susanty(1)        meredita susanty  Benar
2           arief sartono 15           arief sartono  Benar
3        hartanto tantriawan  hartanto tantriawan 1   Benar
4           arief sartono 10           arief sartono  Benar
..                       ...                     ...    ...
101           baud prananto6         yaya setiyadi 1  Salah
102  hartanto tantriawan (1)  hartanto tantriawan 1   Benar
103      dewi tresnawati (1)         dewi tresnawati  Benar
104        imam ekowicaksono    imam ekowicaksono GB  Benar
105             meza silvana         meza silvana GB  Benar

[106 rows x 3 columns]
akurasi: 82.0754716981132


**Proses Menampilkan Hasil dengan Gambar**

In [ ]:
visual_path=pd.DataFrame(visual)
#Proses menampilkan gambar untuk tiap gambar yang dideteksi
for i in range(len(visual_path)):
  if((pathlib.Path(visual_path[0][i]).suffix != '.jfif') and (pathlib.Path(visual_path[1][i]).suffix != '.jfif')):
    print(i)
    pathsumber='/content/drive/MyDrive/tugasFR/tes/'+visual_path[0][i]
    labelsumber=visual_path[0][i]
    display(Image(filename=pathsumber,width = 100, height = 100))
    print(pathlib.Path(labelsumber).stem)
    pathtarget=visual_path[1][i]
    labeltarget=visual_path[1][i]
    display(Image(filename=pathtarget,width = 100, height = 100))
    print(pathlib.Path(labeltarget).stem)
    print(hasil["Hasil Deteksi"][i])
    print('-------------------------------------------------------')